# Computing the Carbon Footprint of a Kaggle Competition

In [1]:
import pandas as pd

In [2]:
PATH = "results/"

# Kaggle Competition: Team 2

## Pipeline1: Pretraining

In [65]:
# This is for 1 fold, out of 5
# This will be used to find OUR carbon footprint
pretrain_1_file = PATH+"siim_pretrain_results_1epoch.csv"
ptrain_1 = pd.read_csv(pretrain_1_file)

#ptrain

In [66]:
# Fill in NaNs with average of the other 3 columns
ptrain_1['energy(kWh)'] = ptrain_1['energy(kWh)'].fillna(ptrain_1['energy(kWh)'].mean())
ptrain_1['CO2eq(g)'] = ptrain_1['CO2eq(g)'].fillna(ptrain_1['CO2eq(g)'].mean())
ptrain_1['distance(km)'] = ptrain_1['distance(km)'].fillna(ptrain_1['distance(km)'].mean())

ptrain_1

,training_file,time,energy(kWh),CO2eq(g),distance(km)
0,cait_pretraining,8:00:33,2.800010,299.601022,2.786986
1,f1_pretraining,11:55:04,4.303103,218.651446,2.033967
2,f3_pretraining,22:14:23,8.025735,229.189462,2.131995
3,l1_pretraining,NaN,5.042949,249.147310,2.317649
4,l1b_pretraining,NaN,5.042949,249.147310,2.317649
5,l2_pretraining,NaN,5.042949,249.147310,2.317649
6,n_cf2_pretraining,NaN,5.042949,249.147310,2.317649


In [67]:
# This is for all 5 folds, the estimated cost of running the full code
pretrain_full_file = PATH+"siim_pretrain_results_full.csv"
ptrain_full = pd.read_csv(pretrain_full_file)

#ptrain_full

In [68]:
ptrain_full['energy(kWh)'] = ptrain_full['energy(kWh)'].fillna(ptrain_full['energy(kWh)'].mean())
ptrain_full['CO2eq(g)'] = ptrain_full['CO2eq(g)'].fillna(ptrain_full['CO2eq(g)'].mean())
ptrain_full['distance(km)'] = ptrain_full['distance(km)'].fillna(ptrain_full['distance(km)'].mean())

ptrain_full

,training_file,time,energy(kWh),CO2eq(g),distance(km)
0,cait_pretraining,40:02:47,14.000048,1126.210028,10.476372
1,f1_pretraining,59:35:21,21.515517,1024.948272,9.534403
2,f3_pretraining,111:11:54,40.128676,1194.298670,11.109755
3,l1_pretraining,NaN,25.214747,1115.152323,10.373510
4,l1b_pretraining,NaN,25.214747,1115.152323,10.373510
5,l2_pretraining,NaN,25.214747,1115.152323,10.373510
6,n_cf2_pretraining,NaN,25.214747,1115.152323,10.373510


## Pipeline1: Training

In [69]:
# This is for 1 fold, out of 5
# This will be used to find OUR carbon footprint
train_1_file = PATH+"siim_train_results_1epoch.csv"
train_1 = pd.read_csv(train_1_file)

#train_1

In [70]:
train_1['energy(kWh)'] = train_1['energy(kWh)'].fillna(train_1['energy(kWh)'].mean())
train_1['CO2eq(g)'] = train_1['CO2eq(g)'].fillna(train_1['CO2eq(g)'].mean())
train_1['distance(km)'] = train_1['distance(km)'].fillna(train_1['distance(km)'].mean())

train_1

,training_file,time,energy(kWh),CO2eq(g),distance(km)
0,n_cf11,1:38:42,0.194869,9.520723,0.088565
1,n_cf11_1,7:33:10,0.578794,25.746986,0.239507
2,n_cf11_9,1:24:16,0.189512,10.865354,0.101073
3,n_cf11_10,1:50:25,0.231844,8.607216,0.080067
4,n_cf11_rot1,NaN,0.298755,13.685070,0.127303
5,n_cf11_6,NaN,0.298755,13.685070,0.127303
6,n_cf11_7,NaN,0.298755,13.685070,0.127303


In [71]:
# This is for all 5 folds, the estimated cost of running the full code
train_full_file = PATH+"siim_train_results_full.csv"
train_full = pd.read_csv(train_full_file)

#train_full

In [72]:
train_full['energy(kWh)'] = train_full['energy(kWh)'].fillna(train_full['energy(kWh)'].mean())
train_full['CO2eq(g)'] = train_full['CO2eq(g)'].fillna(train_full['CO2eq(g)'].mean())
train_full['distance(km)'] = train_full['distance(km)'].fillna(train_full['distance(km)'].mean())

train_full

,training_file,time,energy(kWh),CO2eq(g),distance(km)
0,n_cf11,8:13:32,0.974343,42.493534,0.395289
1,n_cf11_1,37:45:51,2.893969,168.435306,1.566840
2,n_cf11_9,7:01:21,0.947560,50.139455,0.466414
3,n_cf11_10,9:12:03,1.159221,34.338704,0.319430
4,n_cf11_rot1,NaN,1.493773,73.851750,0.686993
5,n_cf11_6,NaN,1.493773,73.851750,0.686993
6,n_cf11_7,NaN,1.493773,73.851750,0.686993


## Calculating Our Carbon Footprint

In [ ]:
def calc_footprint(ptrain_df, train_df, ptrain_runs=1, train_runs=1):
    """
    Function to make a dataframe containing the carbon footprint
    of each state in the training.
    ------------------------------
    Parameters
    ptrain_df: the pretraining dataframe
    train_df: the training dataframe
    ptrain_runs: the number of times we ran pretraining
    train_runs: the number of times we ran training
    """
    # Make new df
    siim_fp = pd.DataFrame(columns=["stage","CO2eq(g)","distance(km)"])
    
    # Compute Sums
    ptrain_co2sum = ptrain_df["CO2eq(g)"].sum() * ptrain_runs       # MULTIPLY BY NUMBER OF TIMES RAN
    ptrain_dist_sum = ptrain_df["distance(km)"].sum() * ptrain_runs # MULTIPLY BY NUMBER OF TIMES RAN

    train_co2sum = train_df["CO2eq(g)"].sum() * train_runs        # MULTIPLY BY NUMBER OF TIMES RAN
    train_dist_sum = train_df["distance(km)"].sum() * train_runs  # MULTIPLY BY NUMBER OF TIMES RAN

    # Make new rows
    row1 = pd.DataFrame({"stage":"ptrain", "CO2eq(g)":ptrain_co2sum, "distance(km)":ptrain_dist_sum}, index=[0])
    row2 = pd.DataFrame({"stage":"train", "CO2eq(g)":train_co2sum, "distance(km)":train_dist_sum}, index=[1])

    # Add to df
    siim_fp = pd.concat([row1, siim_fp.loc[:]]).reset_index(drop=True)
    siim_fp = pd.concat([row2, siim_fp.loc[:]])

    return siim_fp

In [76]:
# Carbon footprint of 1 epoch
calc_footprint(ptrain_1, train_1, 1, 1)

,stage,CO2eq(g),distance(km)
1,train,95.795488,0.891121
0,ptrain,1744.031170,16.223545


# Calculating Team 2's Carbon Footprint

In [92]:
# Carbon footprint of the full training, 
team2_totals = calc_footprint(ptrain_full, train_full, 1, 1)
team2_totals

,stage,CO2eq(g),distance(km)
1,train,516.962248,4.808953
0,ptrain,7806.066263,72.614570


# RevisitingTransfer

## Base: ImageNet Freeze=False

In [83]:
# These results are for the number of epochs in the column "epochs"
# The column "class_epochs" is the number of epochs during training the classification layer
# we need to figure out the cost per single epoch
# then compute the estimated cost for all of the training (including training of the classification layer)
rt_frz_false_file = PATH+"rt_imagenet_frzfalse_results.csv"
rt_frz_false = pd.read_csv(rt_frz_false_file)

rt_frz_false

,dataset,time,energy(kWh),CO2eq(g),distance(km),epochs,class_epochs
0,isic,1:37:40,0.338987,32.058499,0.298219,4,11
1,breast,0:31:18,0.119732,4.190629,0.038983,4,8
2,chest,1:15:20,0.260203,9.107116,0.084717,4,12
3,knee,0:49:39,0.201474,7.353786,0.068407,4,6
4,thyroid,0:14:25,0.009719,0.874699,0.008137,4,4


In [84]:
# Calculate total epochs ran
rt_frz_false["total_epochs"] = rt_frz_false["epochs"] + rt_frz_false["class_epochs"]

# Get CO2eq and distance per single epoch
rt_frz_false["CO2eq(g)/e"] = rt_frz_false["CO2eq(g)"] / rt_frz_false["epochs"]
rt_frz_false["distance(km)/e"] = rt_frz_false["distance(km)"] / rt_frz_false["epochs"]

# Get Total CO2eq and distance per model
rt_frz_false["total_CO2eq(g)"] = rt_frz_false["CO2eq(g)/e"] * rt_frz_false["total_epochs"]
rt_frz_false["total_distance(km)"] = rt_frz_false["distance(km)/e"] * rt_frz_false["total_epochs"]

rt_frz_false[["dataset","total_CO2eq(g)","total_distance(km)"]]

,dataset,total_CO2eq(g),total_distance(km)
0,isic,120.219371,1.118321
1,breast,12.571887,0.116949
2,chest,36.428464,0.338868
3,knee,18.384465,0.171017
4,thyroid,1.749398,0.016274


## Base: ImageNet Freeze=True

In [85]:
# These results are for the number of epochs in the column "epochs"
# The column "class_epochs" is the number of epochs during training the classification layer
# we need to figure out the cost per single epoch
# then compute the estimated cost for all of the training (including training of the classification layer)
rt_frz_true_file = PATH+"rt_imagenet_frztrue_results.csv"
rt_frz_true = pd.read_csv(rt_frz_true_file)
rt_frz_true["total_epochs"] = rt_frz_true["epochs"] + rt_frz_true["class_epochs"]


rt_frz_true

,dataset,time,energy(kWh),CO2eq(g),distance(km),epochs,class_epochs,total_epochs
0,isic,1:34:43,0.063648,2.764155,0.025713,4,7,11
1,breast,0:22:41,0.014810,1.443971,0.013432,4,10,14
2,chest,0:51:27,0.033632,3.497739,0.032537,4,11,15
3,knee,0:35:19,0.023113,2.126414,0.019781,4,5,9
4,thyroid,0:21:09,0.014226,0.633041,0.005889,4,6,10


In [86]:
# Calculate total epochs ran
rt_frz_true["total_epochs"] = rt_frz_true["epochs"] + rt_frz_true["class_epochs"]

# Get CO2eq and distance per single epoch
rt_frz_true["CO2eq(g)/e"] = rt_frz_true["CO2eq(g)"] / rt_frz_true["epochs"]
rt_frz_true["distance(km)/e"] = rt_frz_true["distance(km)"] / rt_frz_true["epochs"]

# Get Total CO2eq and distance per model
rt_frz_true["total_CO2eq(g)"] = rt_frz_true["CO2eq(g)/e"] * rt_frz_true["total_epochs"]
rt_frz_true["total_distance(km)"] = rt_frz_true["distance(km)/e"] * rt_frz_true["total_epochs"]

rt_frz_true[["dataset","total_CO2eq(g)","total_distance(km)"]]

,dataset,total_CO2eq(g),total_distance(km)
0,isic,7.601426,0.070711
1,breast,5.053898,0.047012
2,chest,13.116521,0.122014
3,knee,4.784432,0.044507
4,thyroid,1.582602,0.014722


# Average Cost to Train a Single Model

In [95]:
# CO2 cost
total_models_trained = len(rt_frz_false) + len(rt_frz_true) + 1
CO2_cost_per_model = (rt_frz_true["total_CO2eq(g)"].sum() + rt_frz_false["total_CO2eq(g)"].sum() + team2_totals["CO2eq(g)"].sum()) / total_models_trained
print(f"The average cost of training a single model is {round(CO2_cost_per_model,2)}g CO2eq")

The average cost of training a single model is 776.77g CO2eq


In [101]:
# distance cost
dist_cost_per_model = (rt_frz_true["total_distance(km)"].sum() + rt_frz_false["total_distance(km)"].sum() + team2_totals["distance(km)"].sum()) / total_models_trained
print(f"The average distance cost of training a single model is {round(dist_cost_per_model,2)}km")

The average distance cost of training a single model is 7.23km


# Carbon Footprint of the Whole Competition

Total number of entries from Veronika's supplementary [information](https://static-content.springer.com/esm/art%3A10.1038%2Fs41746-022-00592-y/MediaObjects/41746_2022_592_MOESM1_ESM.pdf).

In [97]:
total_entries = 32307
whole_comp_cost = total_entries * CO2_cost_per_model
print(f"The estimated carbon footprint of the whole competition is {round(whole_comp_cost,2)}g CO2eq")

The estimated carbon footprint of the whole competition is 25095258.11g CO2eq


In [99]:
print(f"Which is equivalent to {round(275787.835915,2)}lbs of CO2")

Which is equivalent to 275787.84lbs of CO2


In [102]:
whole_comp_dist = total_entries * dist_cost_per_model
print(f"The estimated carbon footprint of the whole competition in distance driven {round(whole_comp_dist,2)}km")

The estimated carbon footprint of the whole competition in distance driven 233444.27km
